In [318]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime, os



In [319]:
df_files = os.listdir('../../data/DataGithub/')
df_files

['AntoineAugusti_Vacances_19902026.csv',
 'AntoineAugusti_Vacances_Table_19902026.csv',
 'AntoineAugusti_Vacances_Table_19902026_nonCorrigé.csv',
 'code_etablissement.csv',
 'Effectifs_ecoles.csv',
 'fetes_chretiennes.csv',
 'fetes_juives.csv',
 'fetes_musulmanes.csv',
 'frequentation_cantines_v3.csv',
 'incidence RDD 3.csv',
 'incidence RDD 6.csv',
 'incidence RDD 7.csv',
 'Journees_de_greve.csv',
 'jours-feries-seuls.csv',
 'key_jours.xlsx',
 'Liste ETS_geo_AE12102020.xlsx',
 'menus_2011-2015.csv',
 'menus_2016-2019.csv',
 'missing_strikes.xlsx',
 'mouvements-sociaux-depuis-2002.csv',
 'mouvements-sociaux-depuis-2002.json',
 'orga_des_etablissements.xlsx',
 'ramadan.csv',
 'vacances_Nantes_2011-2019.csv']

In [320]:
df = pd.read_csv('244400404_effectifs-eleves-cleaned')
df['datetime']= df['Date'].apply( lambda string1 : datetime.datetime.strptime(string1,"%Y-%m-%d"))
df['joursem']= df['datetime'].apply( lambda date1 : date1.weekday())
df['isoweek']= df['datetime'].apply( lambda date1 : date1.isocalendar().week)
df.head()

,RowNo,Type de site,Date,Prévision,Réel,Nom du site,An,RéelMed,RéelNorm,datetime,joursem,isoweek
0,0,M/E,2011-09-06,290.0,271.0,LEDRU ROLLIN/SARAH BERNHARDT,2011,308.0,0.879870,2011-09-06,1,36
1,1,M/E,2011-09-05,95.0,102.0,BEAUJOIRE,2011,129.0,0.790698,2011-09-05,0,36
2,2,M/E,2011-09-05,167.0,171.0,CONTRIE,2011,211.0,0.810427,2011-09-05,0,36
3,3,M,2011-09-05,80.0,64.0,COUDRAY M.,2011,77.0,0.831169,2011-09-05,0,36
4,4,M,2011-09-05,73.0,77.0,FONTENY,2011,92.0,0.836957,2011-09-05,0,36


In [321]:
df_calend=pd.read_csv('../../data/SiteNantes/CSV/244400404_nombre-convives-jour-cantine-nantes-2011.csv', sep=';')
df_calend['datetime']= df_calend['Date'].apply( lambda string1 : datetime.datetime.strptime(string1,"%Y-%m-%d")) 
df_calend=df_calend['datetime'].unique()
df_calend = np.sort(df_calend)
df_calend.reshape(-1,1)
df_calend

array(['2011-01-03T00:00:00.000000000', '2011-01-04T00:00:00.000000000',
       '2011-01-06T00:00:00.000000000', ...,
       '2022-05-20T00:00:00.000000000', '2022-05-23T00:00:00.000000000',
       '2022-05-24T00:00:00.000000000'], dtype='datetime64[ns]')

In [322]:
df_calend3 = pd.DataFrame(df_calend)
df_calend3.rename(columns={0: 'datetime'}, inplace=True)
df_calend3['joursem']= df_calend3['datetime'].apply( lambda date1 : date1.weekday())
df_calend3['isoweek']= df_calend3['datetime'].apply( lambda date1 : date1.isocalendar().week)
df_calend3['an']= df_calend3['datetime'].apply( lambda date1 : date1.year)
# df_calend3.head()

In [323]:
df_ponts=[[df_calend[i],int(min(abs(df_calend[i]-df_calend[i-1]),abs(df_calend[i+1]-df_calend[i]))/np.timedelta64(1, "D")-1)] for i in range(1,len(df_calend)-1) ]
df_ponts=pd.DataFrame(df_ponts)
df_ponts.rename(columns={0: 'datetime', 1: 'datelag'}, inplace=True)

In [324]:
df_lagbefore=[[df_calend[i],int(abs(df_calend[i]-df_calend[i-1])/np.timedelta64(1, "D"))-1] for i in range(1,len(df_calend)) ]
df_lagbefore=pd.DataFrame(df_lagbefore)
df_lagbefore.rename(columns={0: 'datetime', 1: 'lagbefore'}, inplace=True)

In [325]:
df_lagafter=[[df_calend[i],int(abs(df_calend[i+1]-df_calend[i])/np.timedelta64(1, "D"))-1] for i in range(0,len(df_calend)-1) ]
df_lagafter=pd.DataFrame(df_lagafter)
df_lagafter.rename(columns={0: 'datetime', 1: 'lagafter'}, inplace=True)

In [326]:
df_calend3=pd.merge(df_calend3, df_ponts, how='left', on='datetime')
df_calend3=pd.merge(df_calend3, df_lagbefore, how='left', on='datetime')
df_calend3=pd.merge(df_calend3, df_lagafter, how='left', on='datetime')

df_calend3['datelag'].fillna(0, inplace=True)
df_calend3['lagbefore'].fillna(0, inplace=True)
df_calend3['lagafter'].fillna(0, inplace=True)

In [327]:
df_calend3.head()

,datetime,joursem,isoweek,an,datelag,lagbefore,lagafter
0,2011-01-03,0,1,2011,0.0,0.0,0.0
1,2011-01-04,1,1,2011,0.0,0.0,1.0
2,2011-01-06,3,1,2011,0.0,1.0,0.0
3,2011-01-07,4,1,2011,0.0,0.0,2.0
4,2011-01-10,0,2,2011,0.0,2.0,0.0


In [328]:
df_calend_official = pd.read_csv('../../data/DataGithub/AntoineAugusti_Vacances_Table_19902026.csv',encoding='latin1',sep=';')
df_calend_official.head()

,An,Fin_Noel,Deb_Hiver,Fin_Hiver,Deb_Printemps,Fin_Printemps,Deb_Eté,Fin_Eté,Deb_Toussaint,Fin_Toussaint,Deb_Noel
0,1991.0,06/01/1991,16/02/1991,03/03/1991,20/04/1991,05/05/1991,07/07/1991,09/09/1991,26/10/1991,03/11/1991,21/12/1991
1,1992.0,05/01/1992,15/02/1992,01/03/1992,11/04/1992,26/04/1992,09/07/1992,09/09/1992,24/10/1992,01/11/1992,19/12/1992
2,1993.0,03/01/1993,27/02/1993,14/03/1993,24/04/1993,09/05/1993,08/07/1993,06/09/1993,23/10/1993,01/11/1993,18/12/1993
3,1994.0,02/01/1994,26/02/1994,13/03/1994,23/04/1994,08/05/1994,06/07/1994,07/09/1994,25/10/1994,06/11/1994,20/12/1994
4,1995.0,03/01/1995,11/02/1995,26/02/1995,08/04/1995,23/04/1995,06/07/1995,04/09/1995,26/10/1995,05/11/1995,23/12/1995


In [329]:
df_calend_official=df_calend_official[0:35]

In [330]:
df_calend_official['An'] = df_calend_official['An'].astype(int)

In [331]:
frenchdate = lambda date1 : datetime.datetime.strptime(date1,"%d/%m/%Y")
usadate = lambda string1 : datetime.datetime.strptime(string1,"%Y-%m-%d")

In [332]:
df_calend_official['Fin_Noel'] = df_calend_official['Fin_Noel'].apply(frenchdate)
df_calend_official['Deb_Hiver'] = df_calend_official['Deb_Hiver'].apply(frenchdate)
df_calend_official['Fin_Hiver'] = df_calend_official['Fin_Hiver'].apply(frenchdate)
df_calend_official['Deb_Printemps'] = df_calend_official['Deb_Printemps'].apply(frenchdate)
df_calend_official['Fin_Printemps'] = df_calend_official['Fin_Printemps'].apply(frenchdate)
df_calend_official['Deb_Eté'] = df_calend_official['Deb_Eté'].apply(frenchdate)
df_calend_official['Fin_Eté'] = df_calend_official['Fin_Eté'].apply(frenchdate)
df_calend_official['Deb_Toussaint'] = df_calend_official['Deb_Toussaint'].apply(frenchdate)
df_calend_official['Fin_Toussaint'] = df_calend_official['Fin_Toussaint'].apply(frenchdate)
df_calend_official['Deb_Noel'] = df_calend_official['Deb_Noel'].apply(frenchdate)


In [333]:
df_calend_official.head()

,An,Fin_Noel,Deb_Hiver,Fin_Hiver,Deb_Printemps,Fin_Printemps,Deb_Eté,Fin_Eté,Deb_Toussaint,Fin_Toussaint,Deb_Noel
0,1991,1991-01-06,1991-02-16,1991-03-03,1991-04-20,1991-05-05,1991-07-07,1991-09-09,1991-10-26,1991-11-03,1991-12-21
1,1992,1992-01-05,1992-02-15,1992-03-01,1992-04-11,1992-04-26,1992-07-09,1992-09-09,1992-10-24,1992-11-01,1992-12-19
2,1993,1993-01-03,1993-02-27,1993-03-14,1993-04-24,1993-05-09,1993-07-08,1993-09-06,1993-10-23,1993-11-01,1993-12-18
3,1994,1994-01-02,1994-02-26,1994-03-13,1994-04-23,1994-05-08,1994-07-06,1994-09-07,1994-10-25,1994-11-06,1994-12-20
4,1995,1995-01-03,1995-02-11,1995-02-26,1995-04-08,1995-04-23,1995-07-06,1995-09-04,1995-10-26,1995-11-05,1995-12-23


In [334]:
df_calend_official.set_index('An', inplace=True)

In [335]:
cal_season = lambda date1 : (date1>df_calend_official.loc[date1.year]['Fin_Noel'])\
    + (date1>df_calend_official.loc[date1.year]['Fin_Hiver'])\
    + (date1>df_calend_official.loc[date1.year]['Fin_Printemps'])\
    + (date1>df_calend_official.loc[date1.year]['Fin_Eté'])\
    + (date1>df_calend_official.loc[date1.year]['Fin_Toussaint'])

In [336]:
list_seasons = ['Noel', 'Hiver', 'Printemps', 'Eté', 'Toussaint']

In [337]:
cycle_seasons = lambda x1 : list_seasons[(x1 % 5)-1]

In [338]:
problem_date = lambda date1 : (date1 <= df_calend_official.loc[date1.year]['Fin_Noel'] or date1>=df_calend_official.loc[date1.year]['Deb_Noel'])\
    or (date1>=df_calend_official.loc[date1.year]['Deb_Hiver'] and date1<=df_calend_official.loc[date1.year]['Fin_Hiver'])\
    or (date1>=df_calend_official.loc[date1.year]['Deb_Printemps'] and date1<=df_calend_official.loc[date1.year]['Fin_Printemps'])\
    or (date1>=df_calend_official.loc[date1.year]['Deb_Eté'] and date1<=df_calend_official.loc[date1.year]['Fin_Eté'])\
    or (date1>=df_calend_official.loc[date1.year]['Deb_Toussaint'] and date1<=df_calend_official.loc[date1.year]['Fin_Toussaint'])

In [339]:
df_calend3['Saison'] = df_calend3['datetime'].apply(cal_season)

In [340]:
df_calend3['Problème'] = df_calend3['datetime'].apply(problem_date)

In [341]:
df_calend3[df_calend3['Problème']==True]

,datetime,joursem,isoweek,an,datelag,lagbefore,lagafter,Saison,Problème
56,2011-05-02,0,18,2011,0.0,9.0,0.0,2,True
57,2011-05-03,1,18,2011,0.0,0.0,1.0,2,True
58,2011-05-05,3,18,2011,0.0,1.0,0.0,2,True
59,2011-05-06,4,18,2011,0.0,0.0,2.0,2,True
992,2018-04-23,0,17,2018,0.0,2.0,0.0,2,True
993,2018-04-24,1,17,2018,0.0,0.0,19.0,2,True


In [342]:
df_calend_official.loc[2011,'Fin_' + cycle_seasons(cal_season(frenchdate('09/01/2011')))]

Timestamp('2011-01-02 00:00:00')

In [343]:
df_calend3['durée_DepuisVac'] = df_calend3['datetime'].apply(lambda date1 : (date1 - df_calend_official.loc[date1.year,'Fin_' + cycle_seasons(cal_season(date1))]).days)

In [344]:
df_calend3['durée_AvantVac'] = df_calend3['datetime'].apply(lambda date1 : (df_calend_official.loc[date1.year,'Deb_' + cycle_seasons(cal_season(date1)+1)] - date1).days)

In [347]:
df_calend3.keys()

Index(['datetime', 'joursem', 'isoweek', 'an', 'datelag', 'lagbefore',
       'lagafter', 'Saison', 'Problème', 'durée_DepuisVac', 'durée_AvantVac'],
      dtype='object')

In [348]:
df_calend3[['datetime', 'Saison', 'Problème', 'durée_DepuisVac', 'durée_AvantVac']].to_csv('calendrier_saisons_vacances')